In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

In [2]:
# load dataset
data_in = 'all_beta_center_p001_20190530'
dataset = data_in + '_500PCs'
npc=100
z = np.load('/oak/stanford/groups/mrivas/projects/degas-risk/datasets/all_pop/'+dataset+'.npz')
scores = pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/scorefiles/'+dataset+'_full.profile',
                       sep='\s+',
                       index_col='IID').sort_index()
scores.iloc[:,:10].tail()

,SCORE_PC1,SCORE_PC2,SCORE_PC3,SCORE_PC4,SCORE_PC5,SCORE_PC6,SCORE_PC7,SCORE_PC8,SCORE_PC9,SCORE_PC10
IID,,,,,,,,,,
6026191,-463.3860,11.66510,132.9630,-78.0225,67.21390,30.2758,-67.7192,34.50320,-79.2435,-220.74300
6026202,70.3511,31.31460,103.2000,75.8685,-100.58100,111.0570,70.0325,-42.40850,15.5442,70.22610
6026216,-660.4920,100.07600,48.6095,77.9837,58.71350,-54.3334,56.8444,-9.92401,48.4059,49.95840
6026229,-139.3400,3.40067,38.9860,43.5834,-218.24000,85.8769,37.6119,-12.24080,-58.5987,-8.03507
6026237,-70.9231,73.12170,348.2290,-108.4140,1.62614,86.4163,-63.2820,4.21486,123.6100,46.68830


In [3]:
# load PRS data (includes phenotypes!)
phenos=['INI50','INI21001','HC269','HC382','HC221','HC294','HC326','cancer1060']

# helper
get_f = lambda phe: '/oak/stanford/groups/mrivas/projects/degas-risk/PRS/all_pop/'+data_in+'/'+phe+'_PRS.profile'
# load
data = reduce(lambda x,y: pd.merge(x, y, left_index=True, right_index=True), 
              map(lambda phe: pd.read_table(get_f(phe), 
                                            sep='\s+', 
                                            skiprows=1,
                                            na_values=-9,
                                            usecols=[1,2,5],
                                            names=['IID',phe,phe+'_PRS'],
                                            index_col='IID'), 
                  phenos)).sort_index()
data[[i for i in data.columns if 'INI' not in i]] -= 1
data.tail()

,INI50,INI50_PRS,INI21001,INI21001_PRS,HC269,HC269_PRS,HC382,HC382_PRS,HC221,HC221_PRS,HC294,HC294_PRS,HC326,HC326_PRS,cancer1060,cancer1060_PRS
IID,,,,,,,,,,,,,,,,
6026191,162.0,81.2988,32.5789,94.27250,0.0,-32.03870,0.0,3.58478,0.0,-20.0895,0.0,-2.00039,0.0,9.06200,0.0,-28.22480
6026202,168.0,29.2941,24.1993,-14.45570,0.0,3.55312,0.0,24.51580,0.0,36.1575,0.0,35.35590,0.0,4.10299,0.0,13.64210
6026216,183.0,46.9242,33.6827,119.70100,0.0,-33.27100,0.0,-51.42370,0.0,31.8385,0.0,3.82058,0.0,11.56950,0.0,12.82230
6026229,174.0,19.6134,25.4657,-7.34802,0.0,-25.15460,0.0,13.68410,0.0,-30.1255,0.0,-2.64008,0.0,12.17560,0.0,8.35452
6026237,180.0,-80.7357,32.5617,50.60180,1.0,-8.52000,0.0,47.50080,1.0,29.6865,0.0,23.95540,1.0,-4.43399,0.0,28.21620


In [4]:
# add DEGAS Risk score to data
for phe in phenos:
    # for npc in range(50,501,50):
    pcvec = z['V'][np.where(z['label_phe_code'] == phe),:].flatten()[:npc]
    data[phe+'_DEGAS_{}'.format(npc)] = scores.iloc[:,:npc].dot(pcvec)
data.tail()

,INI50,INI50_PRS,INI21001,INI21001_PRS,HC269,HC269_PRS,HC382,HC382_PRS,HC221,HC221_PRS,...,cancer1060,cancer1060_PRS,INI50_DEGAS_100,INI21001_DEGAS_100,HC269_DEGAS_100,HC382_DEGAS_100,HC221_DEGAS_100,HC294_DEGAS_100,HC326_DEGAS_100,cancer1060_DEGAS_100
IID,,,,,,,,,,,,,,,,,,,,,
6026191,162.0,81.2988,32.5789,94.27250,0.0,-32.03870,0.0,3.58478,0.0,-20.0895,...,0.0,-28.22480,72.980773,106.978317,7.412809,21.625884,26.228259,0.794388,-0.167584,-2.969542
6026202,168.0,29.2941,24.1993,-14.45570,0.0,3.55312,0.0,24.51580,0.0,36.1575,...,0.0,13.64210,17.471392,-22.265933,-2.593742,29.412308,4.116563,0.042519,0.355809,7.685721
6026216,183.0,46.9242,33.6827,119.70100,0.0,-33.27100,0.0,-51.42370,0.0,31.8385,...,0.0,12.82230,53.493474,104.823546,-5.745743,-50.128854,23.824231,2.427706,0.772379,-4.684932
6026229,174.0,19.6134,25.4657,-7.34802,0.0,-25.15460,0.0,13.68410,0.0,-30.1255,...,0.0,8.35452,13.316989,-13.486012,6.054265,10.377910,-52.520156,-1.411722,1.307465,5.963115
6026237,180.0,-80.7357,32.5617,50.60180,1.0,-8.52000,0.0,47.50080,1.0,29.6865,...,0.0,28.21620,-73.499203,51.979805,-1.243773,38.157516,20.932204,-0.677991,4.813622,-0.510494


In [5]:
# populations to subset to
a,b = 'white_british', 'non_british_white'
with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+a+'.phe', 'r') as f:
    wb = set(filter(lambda x:x in data.index, [int(line.split()[0]) for line in f]))

with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+b+'.phe', 'r') as f:
    nbw = set(filter(lambda x:x in data.index, [int(line.split()[0]) for line in f]))
    
print(len(wb), len(nbw))

(337151, 25486)


In [6]:
# within-sample evaluation: r^2 for QTs, AUC for BINs
score_qts = lambda p,m: data.loc[wb,[p,p+'_'+m]].corr().iloc[0,1]**2
score_bin = lambda p,m: roc_auc_score(data.loc[wb,p].astype(bool), data.loc[wb,p+'_'+m])
score = lambda p,m: score_qts(p,m) if 'INI' in p else score_bin(p,m)
models = ['PRS','DEGAS_'+str(npc)]
matt = pd.DataFrame([[score(p,m) for m in models] for p in phenos], columns=models, index=phenos)
matt['CORR'] = matt.index.map(lambda s: data.loc[wb,[s+'_'+m for m in models]].corr().iloc[0,1]**2)

In [7]:
# out of sample evaluation: r^2 for QTs, AUC for BINs
score_qts = lambda p,m: data.loc[nbw,[p,p+'_'+m]].corr().iloc[0,1]**2
score_bin = lambda p,m: roc_auc_score(data.loc[nbw,p].astype(bool), data.loc[nbw,p+'_'+m])
score = lambda p,m: score_qts(p,m) if 'INI' in p else score_bin(p,m)
models = ['PRS','DEGAS_'+str(npc)]
matt2 = pd.DataFrame([[score(p,m) for m in models] for p in phenos], columns=models, index=phenos)
matt2['CORR'] = matt.index.map(lambda s: data.loc[nbw,[s+'_'+m for m in models]].corr().iloc[0,1]**2)

In [8]:
pd.concat((matt,matt2), keys=['WB','NBW'])

PRS  DEGAS_100      CORR
WB  INI50       0.110177   0.112303  0.996591
    INI21001    0.146153   0.147624  0.959414
    HC269       0.722799   0.634562  0.541267
    HC382       0.641083   0.637892  0.978252
    HC221       0.730077   0.666643  0.705322
    HC294       0.915415   0.616327  0.024558
    HC326       0.869517   0.661452  0.100835
    cancer1060  0.775260   0.540441  0.192697
NBW INI50       0.081980   0.082746  0.997042
    INI21001    0.056689   0.055539  0.957913
    HC269       0.590892   0.577654  0.533173
    HC382       0.572641   0.572548  0.976566
    HC221       0.586353   0.579330  0.710336
    HC294       0.516455   0.571767  0.024198
    HC326       0.566767   0.562301  0.092311
    cancer1060  0.598671   0.519050  0.222130